In [17]:
import os
import math
import numpy as np
import datetime as dt
import time
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
import warnings
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

%matplotlib nbagg

warnings.filterwarnings('ignore')

### Read and Initial Clean

In [20]:
data = pd.read_csv('Data/TrainingData.csv', index_col=0)

data.replace('missing', np.nan, inplace=True)
data.replace('na', np.nan, inplace=True)
int_cols = list(set(data.columns) - {'mvar47'})
data[int_cols] = data[int_cols].astype(float)
data.describe()

,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,mvar10,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar48,default_ind
count,79267.000000,77114.000000,82465.000000,82465.000000,82465.000000,63299.000000,7.532600e+04,63291.000000,71318.000000,8.246500e+04,...,76671.000000,17930.000000,25736.000000,80977.000000,82111.000000,74851.000000,37080.000000,59397.000000,83000.000000,83000.000000
mean,1747.511865,1.054816,5.401784,0.461151,1.084012,1633.429280,1.750797e+04,6822.332227,34030.596329,3.022857e+04,...,0.087673,97.851158,72.016570,0.331550,6.553604,0.706595,0.070523,0.093187,556.021277,0.287410
std,94.830127,1.556682,11.091569,1.704292,5.743899,3667.183981,4.677222e+04,10060.346814,50673.150005,6.617798e+04,...,0.423403,29.530657,24.508469,0.373973,6.273107,0.245915,0.290383,0.329054,1106.953295,0.452557
min,1477.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,...,0.000000,0.014000,0.000000,0.000000,0.000000,0.060660,0.000000,0.000000,2.000000,0.000000
25%,1680.000000,0.131800,0.000000,0.000000,0.000000,41.000000,1.750250e+03,496.000000,3542.250000,1.153000e+03,...,0.000000,90.149500,58.174750,0.000000,2.000000,0.515280,0.000000,0.000000,8.000000,0.000000
50%,1743.000000,0.513000,0.300000,0.000000,0.000000,297.000000,7.020500e+03,2507.000000,14389.000000,9.525000e+03,...,0.000000,100.000000,78.839500,0.200000,5.000000,0.739500,0.000000,0.000000,17.000000,0.000000
75%,1813.000000,1.386200,6.595000,0.000000,0.000000,1381.000000,1.811300e+04,9869.500000,44413.000000,3.428800e+04,...,0.000000,111.327000,91.596750,0.571430,10.000000,0.945120,0.000000,0.000000,111.000000,1.000000
max,1950.000000,31.018100,399.334000,25.754000,165.492000,94302.000000,5.637108e+06,291810.000000,840658.000000,5.647073e+06,...,21.000000,631.360000,182.111000,2.000000,91.000000,1.000000,7.000000,5.000000,3247.000000,1.000000


In [21]:
y = data['default_ind']

X_cols = list(data.columns)
X_cols.remove('default_ind')

X = data[X_cols]

X.loc[X['mvar47'] == 'L', 'mvar47'] = 1
X.loc[X['mvar47'] == 'C', 'mvar47'] = 0

In [22]:
categorical = ['mvar47', 'mvar48']
numeric = list(X.columns)

for var in categorical:
    numeric.remove(var)

### Normalize

In [23]:
X_Scaler = StandardScaler()
X_scaled = pd.DataFrame(X_Scaler.fit_transform(X[numeric]), columns=X[numeric].columns, index=X.index)
X_scaled[categorical] = X[categorical]

### Imputation

In [24]:
def basic_impute(data, cols, type_='mean'):
    
    if type_ == 'mean':
        return data.fillna(data[cols].mean()) 
    
    if type_ == 'median':
        return data.fillna(data[cols].median())
    
    if type_ =='mode':
        md = data[cols].mode()
        return data.fillna(md.iloc[0]) 
    
    if type_ == 'CF': #CF - Customer friendly
        imp_vals = data.mean()
        v = [40,31,41,45,35,46,24,16,17,18,12,9,39,2,42,43]
        for i in v:
            imp_vals['mvar'+str(i)] = 0
        med = data.median()
        imp_vals['mvar11'] = med['mvar11']
        
        return data.fillna(imp_vals[cols])    

In [25]:
def KNN_impute(cols_to_impute, X, numeric, train_na_method='mean'):
    
    for col in cols_to_impute:
        
        other_cols = [c for c in numeric if (c != col)]

        X_train = X.loc[~X[col].isna(), other_cols]
        y_train = X.loc[~X[col].isna(), col]

        X_test = X.loc[X[col].isna(), other_cols]
        
        # Impute KNN training data with basci impute
        X_train = basic_impute(X_train, other_cols, type_=train_na_method)
        X_test = basic_impute(X_test, other_cols, type_=train_na_method)
    
        # 0.1 % of the data as neighbours
        model = KNeighborsRegressor(int(0.001*len(X_train)))
        model.fit(X_train, y_train)

        X.loc[X_test.index, col] = model.predict(X_test)
        
    return X

In [40]:
for col in ['mvar3']:

    X_train = X.loc[~X[col].isna()].drop(col, axis=1)
    y_train = X.loc[~X[col].isna(), col]

    X_test = X.loc[X[col].isna()].drop(col, axis=1)

    # Impute KNN training data with basci impute
    X_train = basic_impute(X_train, X_train.columns, type_='CF')
    X_test = basic_impute(X_test, X_test.columns, type_='CF')

#     # 0.1 % of the data as neighbours
    model = KNeighborsRegressor(int(0.001*len(X_train)))
    model.fit(X_train, y_train)

#     X.loc[X_test.index, col] = model.predict(X_test)

In [ ]:
cols_to_impute = ['mvar9'] # Choose columns you want to impute using the KNN method

X_scaled_imputed = KNN_impute(cols_to_impute, X_scaled, numeric, train_na_method='CF')

### Remove Outliers

In [ ]:
### Remove Outliers

### Feature Engineering

#### Variance Inflation Factor

In [ ]:
VIFs = {}

for var in data.columns:

    ### Remove this after encoding is done
    
    if var in ['mvar48']:
        continue
    
    #######################################
    
    var_y = data[var]
    var_X = data.drop(var, axis=1)
    
    linmod = LinearRegression()
    linmod.fit(var_X, var_y)
    R2 = linmod.score(var_X, var_y)
    VIF[var] = 1 / (1 - R2)

plt.figure()
plt.bar(range(len(VIFs)), VIFs.values(), width=0.7)
plt.ylabel('VIF')
plt.xlabel('Variable')
plt.title('VIF of Variables')
plt.xticks(range(len(VIFs)), VIFs.keys())
plt.show()

#### Correlation

In [ ]:
cov_mat = np.round(np.cov(np.array(X_final[numeric]).T), 3)

fig, ax = plt.subplots(figsize=(40, 40))
img = ax.matshow(cov_mat, cmap='Reds')
fig.colorbar(img, aspect=50)
ax.set_title('Covariance')
ax.set_xticks(ticks=range(len(numeric)))
ax.set_yticks(ticks=range(len(numeric)))

for (i, j), z in np.ndenumerate(cov_mat):
    ax.text(j, i, '{:0.3f}'.format(z), ha='center', va='center')

#### Principal Component Analysis

#### Linear Discriminant Analysis